In [5]:
import os
current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir,"books","智慧型手機使用手冊.txt")
persistent_directory = os.path.join(current_dir,"db","chroma_db_v3")

print(file_path)

try:
    if os.path.exists(persistent_directory):
        if not os.path.exists(file_path):
            raise FileNotFoundError(
                f"檔案{file_path}不存在,請檢查路徑"
            )

        print("檔案正常")
    else:
        print("向量資料庫已經存在,不需要初始化")
except Exception as error:
    print(error)        

c:\Users\user\Documents\github\ai_langchain\lesson7\books\智慧型手機使用手冊.txt
向量資料庫已經存在,不需要初始化


In [9]:
import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import chromadb

current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "books", "智慧型手機使用手冊.txt")
persistent_directory = os.path.join(current_dir, "db", "chroma_db_v3")
try:
    if not os.path.exists(persistent_directory):
        if not os.path.exists(file_path):
            raise FileNotFoundError(
                f"檔案{file_path}不存在,請檢查路徑"
            )

        loader = TextLoader(file_path=file_path,encoding="utf-8")
        documents = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
        docs = text_splitter.split_documents(documents)
        #print(len(docs))
        #print(docs[0].page_content)
        embeddings = HuggingFaceEmbeddings(
            model_name="BAAI/bge-m3"
        )

        client = chromadb.PersistentClient(path=persistent_directory)
        db = Chroma.from_documents(
            docs,
            embedding=embeddings,
            client = client,            
            collection_name="smartphone_manual"
        )




    else:
        print("向量資料庫已經存在,不需要初始化")
except Exception as error:
    print(error)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\user\Documents\github\ai_langchain\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [13]:
query = "支援的 SIM 卡類型?"
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

relevant_docs = retriever.invoke(query)
for doc in relevant_docs:
    print(doc.page_content)
    print("=" * 60)

1.2 SIM 卡安裝
1. 找到手機側邊的 SIM 卡槽
2. 使用退卡針插入卡槽旁的小孔
3. 輕輕推入直到卡槽彈出
4. 將 SIM 卡放入卡槽(注意缺角方向)
5. 將卡槽推回手機
6. 等待 5-10 秒識別 SIM 卡

支援的 SIM 卡類型:
- Nano-SIM(4FF)標準
- 雙卡雙待功能
- 5G/4G LTE 網路

第二章:基本操作

2.1 螢幕手勢
- 點擊:輕觸一次開啟應用程式
- 長按:顯示更多選項或移動圖示
- 滑動:切換畫面或捲動內容
- 雙指縮放:放大或縮小圖片/網頁
- 從上往下滑:開啟通知中心
- 從下往上滑:顯示常用功能快捷鍵

2.2 主畫面配置
- 狀態列:顯示時間、電量、訊號強度
- 通知圖示區:顯示未讀訊息、來電等
- 應用程式圖示:點擊開啟應用程式
- 底部導航列:返回、首頁、多工

自訂主畫面:
1. 長按桌面空白處
2. 選擇「新增小工具」或「桌布」
3. 拖曳應用程式圖示來重新排列
4. 建立資料夾:將一個 App 拖到另一個 App 上
8.4 聯絡客服
客服管道:
- 客服專線:0800-123-456(週一至週日 9:00-21:00)
- 線上客服:官網即時聊天
- 電子郵件:support@example.com
- 服務中心:請至官網查詢鄰近服務中心地址

送修前準備:
1. 備份所有資料
2. 移除 SIM 卡與記憶卡
3. 準備購買證明(發票或收據)
4. 記錄故障現象與發生時間

規格總覽

硬體規格:
- 處理器:Snapdragon 8 Gen 2 / Apple A17 Pro
- 記憶體:8GB/12GB RAM
- 儲存空間:128GB/256GB/512GB
- 螢幕:6.7" AMOLED,120Hz
- 電池:5000mAh
- 重量:195g
- 尺寸:163.1 x 75.5 x 8.2 mm

相機:
- 後置:50MP + 12MP + 8MP
- 前置:32MP

連線:
- 5G、4G LTE
- Wi-Fi 6、藍牙 5.3
- NFC、GPS
- USB-C

作業系統:
- Android 14 或 iOS 17
- 承諾 4 年系統更新
- 5 年安全性更新
相機:
- 後置:50MP + 12MP + 8MP
- 前置:32MP

連線:
- 5G